In [1]:
# Set GPUs
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
# Data Loading

# Import packages
import pandas as pd

# Import data
df = pd.read_csv("../Data/subject-info-cleaned-with-prognosis-D.csv")

# Get prognosis and outcome
df = df[['Prognosis', 'Outcome']]

# Map labels to integers
# Survivor = 0, SCD = 1, PFD = 2
label_map = {"survivor": 0, "sudden cardiac death": 1, "pump failure death": 2}
df['Outcome'] = df['Outcome'].map(label_map)
df.rename(columns = {"Outcome": "labels"}, inplace = True)
df.head()

# Shuffle dataset
# df = df.sample(frac = 1, random_state = 42)
# df.head()

,Prognosis,labels
0,RANKING: 1. Survivor\nREASONING: \n\nBased on ...,0
1,RANKING: 1. Survivor\nREASONING:\n\nBased on t...,0
2,RANKING: 1. Survivor\nREASONING: The patient i...,0
3,RANKING: \n1. Survivor\nREASONING:\nBased on t...,0
4,RANKING: 1. Survivor\nREASONING: \nBased on th...,0


In [6]:
print(df['Prognosis'][810])

RANKING: 
1. Pump failure death
2. Sudden cardiac death
3. Survivor

REASONING:

Based on the provided patient data, I would rank the prognoses from most likely to least likely as follows:

Pump failure death is the most likely outcome in this patient. The patient has a history of Hypertrophic cardiomyopathy, which is a condition that can lead to left ventricular dysfunction and heart failure. The patient's LVEF of 40.0 indicates significant left ventricular dysfunction, which is a strong predictor of heart failure. Additionally, the patient's elevated creatinine level (115.0 mmol/L) suggests impaired renal function, which is often associated with heart failure. The presence of ventricular extrasystoles on the ECG, which is a marker of ventricular instability, further supports this prediction.

Sudden cardiac death is the second most likely outcome. The patient's ECG shows polymorphic ventricular extrasystoles, which can be a marker of ventricular instability and an increased risk of s

In [7]:
# Load in BioBERT with classification head
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import cuda

model_name = "dmis-lab/biobert-base-cased-v1.1"
num_labels = 3  # Three possible outcomes

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

device = 'cuda' if cuda.is_available() else 'cpu'
model.to(device)

/home/sswee/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# Data split (for testing)
from sklearn.model_selection import train_test_split

# Split data
train_df, test_df = train_test_split(df, test_size = 0.3, random_state = 42, stratify = df['labels'])

# Get encodings
train_encodings = tokenizer(list(train_df['Prognosis']), truncation = True, padding = True)
test_encodings = tokenizer(list(test_df['Prognosis']), truncation = True, padding = True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)


In [10]:
# Load BioBERT tokenizer
# model_name = "dmis-lab/biobert-base-cased-v1.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

model_name = "dmis-lab/biobert-base-cased-v1.1"
num_labels = 3  # Three possible outcomes

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Prognosis"], padding="max_length", truncation=True, max_length = 512)

# Apply tokenization
dataset = dataset.map(tokenize_function, batched=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 849/849 [00:00<00:00, 1671.29 examples/s]


In [11]:
# Split data
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test["train"]
val_dataset = train_test["test"]


In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Convert logits to class predictions

    # Compute accuracy
    acc = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")

    # Compute AUC (only if there are at least 2 classes)
    # auc = roc_auc_score(labels, logits, multi_class="ovr") if len(set(labels)) > 1 else 0

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
        #"auc": auc
    }


In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="../Results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="../logs",
    logging_steps=10,
    load_best_model_at_end=True,
)


/home/sswee/myenv/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()


/tmp/ipykernel_4061917/360158454.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.485500,0.391664,0.888235,0.896955,0.888235,0.872773
2,0.451000,0.379228,0.847059,0.760660,0.847059,0.800151
3,0.339100,0.343995,0.905882,0.904046,0.905882,0.898669


/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/sswee/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vecto

TrainOutput(global_step=129, training_loss=0.41489963198817054, metrics={'train_runtime': 81.5779, 'train_samples_per_second': 24.97, 'train_steps_per_second': 1.581, 'total_flos': 535962031911936.0, 'train_loss': 0.41489963198817054, 'epoch': 3.0})

In [15]:
trainer.evaluate()

/home/sswee/myenv/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.34399500489234924,
 'eval_accuracy': 0.9058823529411765,
 'eval_precision': 0.9040459467369037,
 'eval_recall': 0.9058823529411765,
 'eval_f1': 0.8986689270302716,
 'eval_runtime': 0.9122,
 'eval_samples_per_second': 186.362,
 'eval_steps_per_second': 12.059,
 'epoch': 3.0}